# Apache AGE Graph Memory Example

This notebook demonstrates how to use Mem0 with Apache AGE as the graph database backend.

Apache AGE is a PostgreSQL extension that provides graph database functionality, combining the power of PostgreSQL with graph capabilities.

## Prerequisites

1. PostgreSQL 11+ installed
2. Apache AGE extension installed
3. A PostgreSQL database created
4. Required Python packages installed

## Installation

First, install the required packages:

In [ ]:
# Install required packages
!pip install "mem0ai[graph]" apache-age-python psycopg2-binary

## Setup Apache AGE with Docker (Optional)

If you don't have PostgreSQL with Apache AGE installed, you can use Docker:

In [ ]:
# Run this in your terminal to start PostgreSQL with Apache AGE
# docker run --name age-postgres \
#   -e POSTGRES_DB=mem0_db \
#   -e POSTGRES_USER=mem0_user \
#   -e POSTGRES_PASSWORD=mem0_password \
#   -p 5432:5432 \
#   -d apache/age

print("Run the Docker command above in your terminal to start Apache AGE")

## Configuration

Set up your environment variables and configuration:

In [ ]:
import os
from mem0 import Memory

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"

# Apache AGE configuration
config = {
    "embedder": {
        "provider": "openai",
        "config": {"model": "text-embedding-3-large", "embedding_dims": 1536},
    },
    "graph_store": {
        "provider": "apache_age",
        "config": {
            "host": "localhost",
            "port": 5432,
            "database": "mem0_db",
            "username": "mem0_user",
            "password": "mem0_password",
            "graph_name": "movie_graph",
            "base_label": True
        },
    },
}

print("Configuration set up successfully!")

## Initialize Memory with Apache AGE

Create a Memory instance with Apache AGE as the graph store:

In [ ]:
# Initialize Memory with Apache AGE
m = Memory.from_config(config_dict=config)

app_id = "movies"
user_id = "alice"

# Clean up any existing data
m.delete_all(user_id=user_id)

print("Memory initialized with Apache AGE successfully!")

## Adding Memories

Let's add some movie-related memories to see how Apache AGE handles graph relationships:

In [ ]:
# Add movie preferences
messages = [
    {
        "role": "user",
        "content": "I love watching science fiction movies, especially Blade Runner and The Matrix.",
    },
]

result = m.add(messages, user_id=user_id, metadata={"category": "movie_preferences"})
print("Added movie preferences:")
print(result)

In [ ]:
# Add director information
messages = [
    {
        "role": "user",
        "content": "Ridley Scott directed Blade Runner, and the Wachowski sisters directed The Matrix.",
    },
]

result = m.add(messages, user_id=user_id, metadata={"category": "movie_facts"})
print("Added director information:")
print(result)

In [ ]:
# Add actor information
messages = [
    {
        "role": "user",
        "content": "Harrison Ford starred in Blade Runner, and Keanu Reeves starred in The Matrix.",
    },
]

result = m.add(messages, user_id=user_id, metadata={"category": "movie_cast"})
print("Added actor information:")
print(result)

In [ ]:
# Add release year information
messages = [
    {
        "role": "user",
        "content": "Blade Runner was released in 1982, and The Matrix was released in 1999.",
    },
]

result = m.add(messages, user_id=user_id, metadata={"category": "movie_years"})
print("Added release year information:")
print(result)

## Retrieving All Memories

Let's see all the memories and relationships that have been stored:

In [ ]:
# Get all memories
all_results = m.get_all(user_id=user_id)

print("All stored memories:")
for memory in all_results["results"]:
    print(f"- {memory['memory']} [hash: {memory['hash']}]")

print("\nGraph relationships:")
for entity in all_results["entities"]:
    print(f"- {entity['source']} --[{entity['relationship']}]--> {entity['target']}")

## Searching Memories

Now let's search for specific information using natural language queries:

In [ ]:
# Search for movie preferences
search_results = m.search("What movies do I like?", user_id=user_id)

print("Search results for 'What movies do I like?':")
for memory in search_results["results"]:
    print(f"- {memory['memory']}")

print("\nRelated entities:")
for entity in search_results["entities"]:
    print(f"- {entity['source']} --[{entity['relationship']}]--> {entity['target']}")

In [ ]:
# Search for director information
search_results = m.search("Who directed The Matrix?", user_id=user_id)

print("Search results for 'Who directed The Matrix?':")
for memory in search_results["results"]:
    print(f"- {memory['memory']}")

print("\nRelated entities:")
for entity in search_results["entities"]:
    print(f"- {entity['source']} --[{entity['relationship']}]--> {entity['target']}")

In [ ]:
# Search for actor information
search_results = m.search("Which actors starred in these movies?", user_id=user_id)

print("Search results for 'Which actors starred in these movies?':")
for memory in search_results["results"]:
    print(f"- {memory['memory']}")

print("\nRelated entities:")
for entity in search_results["entities"]:
    print(f"- {entity['source']} --[{entity['relationship']}]--> {entity['target']}")

## Advanced: Direct Apache AGE Queries

Apache AGE allows you to run both SQL and Cypher queries. Here's how you can access the underlying connection:

In [ ]:
# Access the graph memory instance to run custom queries
graph_memory = m.memory_store.graph

# Example: Get all nodes and their relationships
cypher_query = """
MATCH (n)-[r]->(m)
WHERE n.user_id = $user_id
RETURN n.name AS source, type(r) AS relationship, m.name AS target
LIMIT 10
"""

try:
    results = graph_memory._execute_cypher(cypher_query, {"user_id": user_id})
    print("Direct Cypher query results:")
    for result in results:
        print(f"- {result['source']} --[{result['relationship']}]--> {result['target']}")
except Exception as e:
    print(f"Error executing query: {e}")

## Cleanup

Clean up the data when you're done:

In [ ]:
# Delete all memories for the user
m.delete_all(user_id=user_id)
print("All memories deleted successfully!")

## Summary

This notebook demonstrated how to:

1. **Configure Mem0 with Apache AGE**: Set up the connection to PostgreSQL with Apache AGE extension
2. **Add Memories**: Store information that gets automatically converted to graph relationships
3. **Search Memories**: Use natural language queries to find relevant information
4. **View Relationships**: See how entities are connected in the graph
5. **Custom Queries**: Run direct Cypher queries on the Apache AGE database

### Benefits of Apache AGE:

- **SQL + Cypher**: Combine relational and graph queries
- **ACID Compliance**: Full transaction support
- **PostgreSQL Ecosystem**: Use existing PostgreSQL tools
- **Scalability**: Proven PostgreSQL performance
- **Cost-Effective**: Open-source solution

### Next Steps:

- Explore more complex graph relationships
- Integrate with your existing PostgreSQL infrastructure
- Use Apache AGE's advanced graph algorithms
- Scale your graph memory system with PostgreSQL clustering